In [1]:

from sim import *

In [2]:
op4 = Operator(4,[],8)
op3 = Operator(3,[4],6,is_join=True)
op2 = Operator(2,[3],4)
op1 = Operator(1,[3],2)
op0 = Operator(0,[1,2],10)

lat = Lattice([op0,op1,op2,op3,op4])

print(lat)
print (lat.get_children(3))

# Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
# [4]

Lattice {0: [1, 2], 1: [3], 2: [3], 3: [4], 4: []}
[4]


``` 
v1 -> v3 -> v4
^      ^   
|      |
v0 ->  v2
```

In [3]:
event_queue = []
e = Event(1,0,0)
e.start(lat,0)
for i in range(10):
    e.step()
e.finish(event_queue,lat,2)
e,event_queue
# (<Event 1; Running Op 0; Available at Time 0; Executed 0 to 2>,
#  [<Event 1; Running Op 1; Available at Time 2; Executed None to None>,
#   <Event 1; Running Op 2; Available at Time 2; Executed None to None>])


(<Event 1; Running Op 0; Available at Time 0; Executed 0 to 3; Deadline: None>,
 [<Event 1; Running Op 1; Available at Time 3; Executed None to None; Deadline: None>,
  <Event 1; Running Op 2; Available at Time 3; Executed None to None; Deadline: None>])

In [4]:
w = Worker(1)
w.do_job(Event(7,4,2),lat,2)
w # Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None; Deadline: None>]

In [5]:
wp = WorkerPool([])
v = Worker(1)
wp.add_workers([w,v])
# ERROR: worker not unique [1]

wp
# Worker Pool: 
# Worker 1: [<Event 7; Running Op 4; Available at Time 2>]

ERROR: worker not unique [1]


Worker Pool: 
Worker 1: [<Event 7; Running Op 4; Available at Time 2; Executed 2 to None; Deadline: None>]

In [6]:
wp1 = WorkerPool([])
wp1.add_worker(v)
wp1

Worker Pool: 
Worker 1: []

In [7]:

simulate(fifo_schedule,[Event(2,0,4),Event(5,0,40)],wp1,lat,100)

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: None>
finished event: <Event 2; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 40; Executed None to None; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 50; Executed 52 to 56; Deadline: None>
finished event: <Event 5; Running Op 

In [8]:
print(wp1.history())

Worker Pool History: 
Worker1: [
  <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
  <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
  <Event 2; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: None>
  <Event 2; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: None>
  <Event 2; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: None>
  <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
  <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
  <Event 5; Running Op 2; Available at Time 50; Executed 52 to 56; Deadline: None>
  <Event 5; Running Op 3; Available at Time 56; Executed 56 to 62; Deadline: None>
  <Event 5; Running Op 4; Available at Time 62; Executed 62 to 70; Deadline: None>
]


In [9]:
wp1.reset()
v.history

[]

In [10]:
x = Worker(2)
x.reset()
wp2 = WorkerPool([])
wp2.add_workers([v,x])
wp2

Worker Pool: 
Worker 1: []
Worker 2: []

In [11]:
simulate(fifo_schedule,[Event(2,0,4),Event(5,0,40)],wp2,lat,100)

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
finished event: <Event 2; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 40; Executed None to None; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 50; Executed 50 to 54; Deadline: None>
finished event: <Event 5; Running Op 

In [12]:
print(wp2.history())

Worker Pool History: 
Worker1: [
  <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
  <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
  <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
  <Event 2; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: None>
  <Event 5; Running Op 0; Available at Time 40; Executed 40 to 50; Deadline: None>
  <Event 5; Running Op 1; Available at Time 50; Executed 50 to 52; Deadline: None>
  <Event 5; Running Op 3; Available at Time 54; Executed 54 to 60; Deadline: None>
  <Event 5; Running Op 4; Available at Time 60; Executed 60 to 68; Deadline: None>
]
Worker2: [
  <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
  <Event 5; Running Op 2; Available at Time 50; Executed 50 to 54; Deadline: None>
]


In [13]:
wp2.reset()
simulate(fifo_schedule,[Event(2,0,4),Event(5,0,4)],wp2,lat,100)
print(wp2.history())

Activate: <Event 2; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
Activate: <Event 5; Running Op 0; Available at Time 4; Executed None to None; Deadline: None>
finished event: <Event 2; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 5; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: None>
finished event: <Event 2; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: None>
finished event: <Event 5; Running Op 1; Available at Time 14; Executed 16 to 18; Deadline: None>
finished event: <Event 2; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: None>
finished event: <Event 5; Running Op 2; Available at Time 14; Executed 18 to 22; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: None>
finished event: <Event 2; Running Op 3; Available at Time 22; Executed 22 to 28; Deadline: None>
finished event: <Event 2; Running Op 4; 

## DEADLINES

In [14]:
d_op4 = Operator(4,[],8, relative_deadline = 10)
d_op3 = Operator(3,[4],6,is_join=True, relative_deadline = 6)
d_op2 = Operator(2,[3],4, relative_deadline=4)
d_op1 = Operator(1,[3],2, relative_deadline=5)
d_op0 = Operator(0,[1,2],10, relative_deadline=10)

d_lat = Lattice([d_op0,d_op1,d_op2,d_op3,d_op4])

In [15]:
wp1.reset()
simulate(fifo_schedule,[Event(1,0,4, deadline = 14)], wp1, d_lat , 100)
print(wp1.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: 18>
finished event: <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
finished event: <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
  <Event 1; Running Op 2; Available at Time 14; Executed 16 to 20; Deadline: 18>
  <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
  <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>

In [16]:
wp.reset()
simulate(edf_schedule,[Event(1,0,4, deadline = 14)], wp, d_lat , 100)
print(wp.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 18 to 20; Deadline: 19>
finished event: <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
finished event: <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
  <Event 1; Running Op 1; Available at Time 14; Executed 18 to 20; Deadline: 19>
  <Event 1; Running Op 3; Available at Time 20; Executed 20 to 26; Deadline: 26>
  <Event 1; Running Op 4; Available at Time 26; Executed 26 to 34; Deadline: 36>

In [17]:
wp2.reset()
simulate(edf_schedule,[Event(1,0,4,14)], wp2, d_lat , 300)
print(wp2.history())

Activate: <Event 1; Running Op 0; Available at Time 4; Executed None to None; Deadline: 14>
finished event: <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
finished event: <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; Deadline: 19>
finished event: <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
finished event: <Event 1; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: 24>
finished event: <Event 1; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: 34>
Worker Pool History: 
Worker1: [
  <Event 1; Running Op 0; Available at Time 4; Executed 4 to 14; Deadline: 14>
  <Event 1; Running Op 2; Available at Time 14; Executed 14 to 18; Deadline: 18>
  <Event 1; Running Op 3; Available at Time 18; Executed 18 to 24; Deadline: 24>
  <Event 1; Running Op 4; Available at Time 24; Executed 24 to 32; Deadline: 34>
]
Worker2: [
  <Event 1; Running Op 1; Available at Time 14; Executed 14 to 16; 